В этом ноубуке проверяем правильность работы предобработки данных

In [8]:
import json
import pandas as pd
import torch
import matplotlib.pyplot as plt

from utils import *
from models.tabddpm_ohe_noise.tabddpm_ohe_noise import *

from tqdm.notebook import tqdm

from eval.base_metrics import calculate_base_metrics
from eval.similarity import calculate_similarity
from eval.mle import calculate_mle
from eval.alpha_beta import calculate_alpha_beta
from eval.detection import calculate_detection
from eval.dcr import calculate_DCR

### Скачиваем данные

In [9]:
dataname = 'x'
model_short = 'x'

In [10]:
from distutils.dir_util import copy_tree
copy_tree("./data/adult", "./data/x")

['./data/x/adult.zip',
 './data/x/Index',
 './data/x/adult.data',
 './data/x/adult.names',
 './data/x/adult.test',
 './data/x/old.adult.names',
 './data/x/test.data',
 './data/x/X_num_train.npy',
 './data/x/X_cat_train.npy',
 './data/x/y_train.npy',
 './data/x/X_num_test.npy',
 './data/x/X_cat_test.npy',
 './data/x/y_test.npy',
 './data/x/train.csv',
 './data/x/test.csv',
 './data/x/info.json',
 './data/x/.ipynb_checkpoints/info-checkpoint.json']

In [11]:
dataname_initial = 'adult'

with open(f'data/Info/{dataname_initial}.json', 'r') as f:
    info = json.load(f)

info['name'] = dataname
info['data_path'] = f'data/{dataname}/{dataname}.data'
info['test_path'] = f'data/{dataname}/{dataname}.test'
info['task_type'] = 'regression'

with open(f'data/Info/{dataname}.json', 'w', encoding='utf-8') as f:
    json.dump(info, f, ensure_ascii=False, indent=4)

download_dataset('adult', dataname)

import os
for file in tqdm(os.listdir(f'./data/{dataname}')):
    old_file = os.path.join(f'./data/{dataname}', file)
    new_file = os.path.join(f'./data/{dataname}', file.replace(dataname_initial, dataname))
    os.rename(old_file, new_file)
    
categorial_to_OHE(dataname)
process_data(dataname)
postprocess_OHE(dataname, dataname_initial)

Start processing dataset adult from UCI.
Aready downloaded.


  0%|          | 0/17 [00:00<?, ?it/s]

{'cat_col_idx': [],
 'cat_col_idx_initial': [6, 7, 8, 9, 10, 11, 12, 13],
 'column_info': {'0': 'float',
                 '10': 'float',
                 '11': 'float',
                 '12': 'float',
                 '13_ ?': 'float',
                 '13_ Cambodia': 'float',
                 '13_ Canada': 'float',
                 '13_ China': 'float',
                 '13_ Columbia': 'float',
                 '13_ Cuba': 'float',
                 '13_ Dominican-Republic': 'float',
                 '13_ Ecuador': 'float',
                 '13_ El-Salvador': 'float',
                 '13_ England': 'float',
                 '13_ France': 'float',
                 '13_ Germany': 'float',
                 '13_ Greece': 'float',
                 '13_ Guatemala': 'float',
                 '13_ Haiti': 'float',
                 '13_ Holand-Netherlands': 'float',
                 '13_ Honduras': 'float',
                 '13_ Hong': 'float',
                 '13_ Hungary': 'float',
        

In [14]:
model_name = 'x'
sample_save_path = f'./synthetic/{dataname}/real.csv'

CONFIG.add_arg('method', model_name)
CONFIG.add_arg('dataname', dataname)

CONFIG.add_arg('real_path',
                   f"synthetic/{CONFIG.get_arg('dataname')}/initial_real.csv")
CONFIG.add_arg('test_path',
                   f"synthetic/{CONFIG.get_arg('dataname')}/initial_test.csv")
CONFIG.add_arg('sample_save_path',
                   f"synthetic/{CONFIG.get_arg('dataname')}/real.csv")


CONFIG.add_arg('info_path',
                   f"./data/{CONFIG.get_arg('dataname')}/initial_info.json")

In [15]:
CONFIG.get_all_args()

{'dataname': 'x',
 'method': 'x',
 'device': 'cuda',
 'mode': 'train',
 'train': 1,
 'sample_save_path': 'synthetic/x/real.csv',
 'sigma_scheduller_name': 'constant',
 'sigma_value': 0.001,
 'num_noise': 103,
 'real_path': 'synthetic/x/initial_real.csv',
 'test_path': 'synthetic/x/initial_test.csv',
 'info_path': './data/x/initial_info.json',
 'save_path': './synthetic/adult_test/initial_tabddpm_test.csv'}

In [16]:
postsample_OHE(dataname, f'./synthetic/{dataname}/initial_{model_name}.csv')

Sample path changed to ./synthetic/x/initial_x.csv


In [17]:
overall_metrics = {}

model_name = 'tabddpm_test'
sample_save_path = f'./synthetic/{dataname}/real.csv'

CONFIG.add_arg('method', model_name)
CONFIG.add_arg('dataname', dataname)
CONFIG.add_arg('sample_save_path',
                   f"synthetic/{CONFIG.get_arg('dataname')}/real.csv")

overall_metrics[model_name] = {}

print(model_name, sample_save_path)

# tabddpm_noise_ohe = TabDDPM_OHE_Noise(CONFIG, model_save_path=model_save_path, sigmas=None)
# tabddpm_noise_ohe.sample(sample_save_path=sample_save_path)
postsample_OHE(dataname, f'./synthetic/{dataname}/initial_{model_name}.csv')

# подсчет метрик
# overall_metrics[model_name]['base_metrics'] = calculate_base_metrics(make_binary=True, value=' >50K') - ошибка
overall_metrics[model_name]['similarity'] = calculate_similarity()
overall_metrics[model_name]['mle'] = calculate_mle()
overall_metrics[model_name]['detection'] = calculate_detection()
overall_metrics[model_name]['DCR'] = calculate_DCR()
overall_metrics[model_name]['quality'] = calculate_alpha_beta()

tabddpm_test ./synthetic/x/real.csv
Sample path changed to ./synthetic/x/initial_tabddpm_test.csv
CALCULATING COLUMN-WISE DENSITY ESTIMATION & PAIR-WISE COLUMN CORRELATION...
Generating report ...

(1/2) Evaluating Column Shapes: |██████████| 15/15 [00:00<00:00, 91.20it/s]|
Column Shapes Score: 100.0%

(2/2) Evaluating Column Pair Trends: |██████████| 105/105 [00:01<00:00, 59.90it/s]|
Column Pair Trends Score: 100.0%

Overall Score (Average): 100.0%

Error rate (%) of column-wise density estimation TABDDPM_TEST: 0.000 ± 0.001
Error rate (%) of pair-wise column correlation score TABDDPM_TEST: 0.000 ± 0.000


DONE! 

DRAW COLUMN VALUES DISTRIBUTIONS...


DONE!


100%|██████████| 36/36 [00:10<00:00,  3.35it/s]


Saving scores to  eval/mle/x/tabddpm_test.json
ROC - AUC обучения на синтетических данных TABDDPM_TEST: 0.918 ± 0.005
x, tabddpm_test: 1.0
DCR Score, a value closer to 0.5 is better
x-tabddpm_test, DCR Score = 0.9992322103129511
=========== All Features ===========
Data shape:  (32561, 110)
alpha precision: 0.999978, beta recall: 0.999978
